In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import sys
import random
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Math
from torch.utils.data import DataLoader, Dataset
import time
import json

from utils import *
from model_utils import *

sys_epsilon = sys.float_info.epsilon

# Data Preprocessing

In this section, we preprocess the CFD data collected from DSM-based LES of flow past a sphere at $\mathcal{R}e=10^3$, $5\times 10^3$, and $10^4$. The data for each Reynolds number is collected from $t=300$ to $320$. From $t=300$ to $310$, data is collected at every ten variable time-steps. However, from $t=310$ to $320$, data is collected with $\Delta t =1$.

In [2]:
paths = ["/home/hmarefat/scratch/torchFOAM/Case_dS/postProcessing/fieldData.dat",
         "/home/hmarefat/scratch/torchFOAM/Case_dS_R53/postProcessing/fieldData.dat",
         "/home/hmarefat/scratch/torchFOAM/Case_dS_R4/postProcessing/fieldData.dat"]
nCols = 26
names = ['_R3', '_R53', '_R4']

In [ ]:
random.seed(35)

count = 1000000
Indx_R3 = returnRandmIndx(4956526, count)
Indx_R53 = returnRandmIndx(5900626, count)
Indx_R4 = returnRandmIndx(4012425, count)

In [ ]:
Indx_R3[0:10]

In [ ]:
random.shuffle(Indx_R3)

In [ ]:
Indx_R3[0:10]

In [ ]:
index_data = []

for indx in [Indx_R3, Indx_R53, Indx_R4]:
    seen, unseen = splitterIndx(indx)
    index_data.append((seen, unseen))

index_array = np.array(index_data, dtype=object)

In [ ]:
index_array[2,0].shape, index_array[2,1].shape

In [ ]:
data = []

for i, path in enumerate(paths):
    name = path.split('/')[-1][:-4]+names[i]
    print(f'Starting to preprocess dataset {name}')
    data.clear()
    
    with open(path, "r") as f:
        next(f)
        for line in f:
            l = line.split()
            try:
                data.append([float(x) for x in l]) #(l)
            except ValueError as e:
                print(f"Error converting line to float: {line.strip()} - {e}")
                continue
                
    #data.pop(0)
    print('Reading raw file is done!')
    ds = np.array(data) 
    
    ds_seen = ds[index_array[i,0]]
    ds_unseen = ds[index_array[i,1]]
    
    scaler(name+'_seen', ds_seen)
    scaler(name+'_unseen', ds_unseen)
    
    print('\n')

# Data Preparation

In [3]:
headers = ["t",                                             # time
           "X", "Y", "Z",                                   # spacial coordinates
           "Ux", "Uy", "Uz",                                # velocity components
           "G1", "G2", "G3", "G4", "G5", "G6",              # velocity gradient tensor components
           "S1", "S2", "S3", "S4", "S5", "S6",              # strain rate tensor compnents
           "UUp1", "UUp2", "UUp3", "UUp4", "UUp5", "UUp6",  # resolved Reynolds stress tensor components
           "Cs"]                                            # Smagorinsky coefficient

### Dataset Loading

#### $\mathcal Re = 10^3$

In [4]:
# Re = 10^3 seen
dSn_R103_seen_means = pd.read_csv('../processedDatasets/fieldData_R3_seen_means.txt', sep=' ', names=headers) 
dSn_R103_seen_scales = pd.read_csv('../processedDatasets/fieldData_R3_seen_scales.txt', sep=' ', names=headers) 
dSn_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen_norm.txt', sep=' ', names=headers)
dS_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen.txt', sep=' ', names=headers)

# Re = 10^3 unseen
dSn_R103_unseen_means = pd.read_csv('../processedDatasets/fieldData_R3_unseen_means.txt', sep=' ', names=headers) 
dSn_R103_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R3_unseen_scales.txt', sep=' ', names=headers) 
dSn_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen_norm.txt', sep=' ', names=headers)
dS_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 5\times 10^3$

In [5]:
# Re = 5 x 10^3 seen
dSn_R503_seen_means = pd.read_csv('../processedDatasets/fieldData_R53_seen_means.txt', sep=' ', names=headers) 
dSn_R503_seen_scales = pd.read_csv('../processedDatasets/fieldData_R53_seen_scales.txt', sep=' ', names=headers) 
dSn_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen_norm.txt', sep=' ', names=headers)
dS_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen.txt', sep=' ', names=headers)

# Re = 5 x 10^3 unseen
dSn_R503_unseen_means = pd.read_csv('../processedDatasets/fieldData_R53_unseen_means.txt', sep=' ', names=headers) 
dSn_R503_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R53_unseen_scales.txt', sep=' ', names=headers) 
dSn_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen_norm.txt', sep=' ', names=headers)
dS_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 10^4$

In [6]:
# Re = 10^4 seen
dSn_R104_seen_means = pd.read_csv('../processedDatasets/fieldData_R4_seen_means.txt', sep=' ', names=headers) 
dSn_R104_seen_scales = pd.read_csv('../processedDatasets/fieldData_R4_seen_scales.txt', sep=' ', names=headers) 
dSn_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen_norm.txt', sep=' ', names=headers)
dS_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen.txt', sep=' ', names=headers)

# Re = 10^4 unseen
dSn_R104_unseen_means = pd.read_csv('../processedDatasets/fieldData_R4_unseen_means.txt', sep=' ', names=headers) 
dSn_R104_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R4_unseen_scales.txt', sep=' ', names=headers) 
dSn_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen_norm.txt', sep=' ', names=headers)
dS_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen.txt', sep=' ', names=headers)

# Model Configuration Setup

In this section, we consider the different model configurations as presented in the following table.

$$
\begin{array}{|l|c|c|c|c|}
    \hline
    \textbf{Model} & \textbf{Inputs} & \textbf{No. of Inputs} & \textbf{Outputs} & \textbf{No. of Outputs} \\
    \hline
    \mathbf{M1} & u_i \, \text{and} \, \mathcal{S}_{ij} & 9 & c_s & 1\\
    \mathbf{M2} & \mathcal{G}_{ij} \, \text{and} \, \mathcal{S}_{ij} & 12 & c_s & 1 \\
    \mathbf{M3} & u_i \, \text{and} \, \tau^{'}_{ij} & 9 & c_s & 1 \\
    \mathbf{M4} & \mathcal{G}_{ij} \, \text{and} \, \tau^{'}_{ij} & 12 & c_s & 1 \\
    \hline
\end{array}
$$

In [7]:
R103 = dSn_R103_seen
R503 = dSn_R503_seen
R104 = dSn_R104_seen

R103_un = dSn_R103_unseen
R503_un = dSn_R503_unseen
R104_un = dSn_R104_unseen

In [8]:
M1_headers = ['Ux', 'Uy', 'Uz', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M2_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M3_headers = ['Ux', 'Uy', 'Uz', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']
M4_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']

M1_103 = R103.filter(M1_headers, axis=1)
M2_103 = R103.filter(M2_headers, axis=1)
M3_103 = R103.filter(M3_headers, axis=1)
M4_103 = R103.filter(M4_headers, axis=1)

M1_503 = R503.filter(M1_headers, axis=1)
M2_503 = R503.filter(M2_headers, axis=1)
M3_503 = R503.filter(M3_headers, axis=1)
M4_503 = R503.filter(M4_headers, axis=1)

M1_104 = R104.filter(M1_headers, axis=1)
M2_104 = R104.filter(M2_headers, axis=1)
M3_104 = R104.filter(M3_headers, axis=1)
M4_104 = R104.filter(M4_headers, axis=1)

M1_103_test = R103_un.filter(M1_headers, axis=1)
M2_103_test = R103_un.filter(M2_headers, axis=1)
M3_103_test = R103_un.filter(M3_headers, axis=1)
M4_103_test = R103_un.filter(M4_headers, axis=1)

M1_503_test = R503_un.filter(M1_headers, axis=1)
M2_503_test = R503_un.filter(M2_headers, axis=1)
M3_503_test = R503_un.filter(M3_headers, axis=1)
M4_503_test = R503_un.filter(M4_headers, axis=1)

M1_104_test = R104_un.filter(M1_headers, axis=1)
M2_104_test = R104_un.filter(M2_headers, axis=1)
M3_104_test = R104_un.filter(M3_headers, axis=1)
M4_104_test = R104_un.filter(M4_headers, axis=1)

# Model Training

In [17]:
dt = M4_103
dt_name = namestr(M4_103, globals())[0]

In [18]:
output_size = 1
input_size = dt.shape[1] - output_size 
neurons_per_layer = [60, 60, 60, 60, 60] 
hidden_layers = len(neurons_per_layer) 

split_sz = 0.8
mask = np.random.rand(len(dt)) < split_sz
train = dt[mask].reset_index(drop=True) 
val = dt[~mask].reset_index(drop=True)

In [19]:
train

,G1,G2,G3,G4,G5,G6,UUp1,UUp2,UUp3,UUp4,UUp5,UUp6,Cs
0,0.179013,-1.056645,0.059678,0.133641,0.681956,0.174060,-0.712982,-0.062303,-0.013639,-0.779575,-0.305482,-0.757006,5.377695
1,-0.009625,-0.032130,0.017649,-0.012303,0.059045,-0.041634,-0.549870,0.205258,-0.137245,-0.407673,-0.680971,-0.568597,-0.518536
2,0.036092,-0.107078,0.150001,-0.029983,-0.129665,0.218191,-0.675243,-0.106069,0.025585,-0.546227,-0.824187,-0.586783,0.014761
3,-0.124293,0.023164,-0.383953,0.083074,-0.033031,-0.427643,1.132239,-1.725099,2.311650,1.591560,-0.375592,1.433032,-0.474977
4,0.283349,0.021385,-0.187639,0.018320,0.030496,0.394027,-0.481485,-0.342886,-0.005546,-0.454776,-0.504209,-0.623866,0.235588
...,...,...,...,...,...,...,...,...,...,...,...,...,...
559565,0.386222,0.586988,-0.052522,0.719699,-0.307615,-0.199605,-0.414118,-0.290010,-0.208508,-0.567458,0.019512,-0.593034,-0.330632
559566,0.053151,0.024931,-0.016800,0.000003,0.034749,0.018040,0.016895,0.670161,0.567272,-0.098345,1.449329,-0.272631,0.216397
559567,-0.082247,0.120722,-0.176876,0.045676,0.006784,-0.247584,-0.570250,-0.249275,0.140189,-0.243682,-0.458520,-0.379598,-0.575114
559568,0.063029,0.249171,-0.168462,0.067508,-0.122395,0.167455,-0.714000,-0.062859,-0.013352,-0.780154,-0.307578,-0.758195,0.228152


In [20]:
val

,G1,G2,G3,G4,G5,G6,UUp1,UUp2,UUp3,UUp4,UUp5,UUp6,Cs
0,0.199419,-0.513504,0.131127,0.344063,-0.875082,-0.698431,-0.062009,0.056643,-0.472813,0.386514,-0.272735,0.245461,0.762858
1,0.364695,-0.219753,0.175665,0.439121,-1.504213,-1.520131,-0.234714,0.198004,0.313033,0.499344,-0.297273,0.443874,-0.337598
2,0.034143,0.101562,-0.030681,0.036390,-0.127633,-0.008348,-0.410421,-0.444282,0.068060,-0.370405,-0.741492,-0.508605,0.211003
3,0.061518,-0.760926,-0.358545,-0.019007,-0.295744,-0.382367,-0.298655,0.085456,-0.559509,-0.382667,-0.530760,-0.210946,0.548835
4,0.094109,-0.110199,-0.175466,0.635107,0.207723,-0.348811,-0.050728,-0.988149,-0.025658,0.188608,-0.180260,-0.015511,-0.125123
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140425,-0.153342,0.046591,-0.202797,0.009197,-0.021060,-0.178875,-0.689692,-0.050586,-0.034499,-0.731649,-0.571031,-0.661013,-0.549827
140426,0.172596,-0.279961,-0.342342,-0.090579,-0.012725,-0.387142,-0.713165,-0.060867,-0.012766,-0.779642,-0.305282,-0.757829,0.335641
140427,1.618197,-1.339991,-0.528364,3.223153,0.699241,-0.114258,3.932996,-2.976239,-2.537021,0.507168,-0.620525,0.824253,-0.494859
140428,1.392144,0.147587,1.591455,-0.654324,-1.196562,5.626526,3.800836,5.478007,3.816482,3.871056,2.179870,3.252265,0.457446


In [21]:
batch_sz_trn = 4096
batch_sz_val = int(batch_sz_trn / 4)

train_dataset = MyDataset(train)
val_dataset = MyDataset(val)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_sz_trn, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_sz_val, shuffle=True)

In [22]:
data_iter = iter(train_loader)
next(data_iter)[0]

tensor([-0.0783, -0.1465,  0.0995, -0.0387,  0.0247, -0.1710, -0.1805, -0.5435,
         0.5049, -0.2325, -0.4339, -0.0593, -0.4817], dtype=torch.float64)

In [23]:
model = MLPModel(input_size=input_size, 
                 output_size=output_size, 
                 hidden_layers=hidden_layers, 
                 neurons_per_layer=neurons_per_layer)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=3, gamma=0.2)

device = torch.device("cuda")
model.to(device)
model.double()

MLPModel(
  (block): Sequential(
    (0): Linear(in_features=12, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=60, bias=True)
    (3): ReLU()
    (4): Linear(in_features=60, out_features=60, bias=True)
    (5): ReLU()
    (6): Linear(in_features=60, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=60, bias=True)
    (9): ReLU()
    (10): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [24]:
epochs = 6000
best_loss = float('inf')
PATH = f"./best_model_{dt_name}.pt"

early_stopper = EarlyStopper(patience=300, path=PATH)
history = {
    "train_loss": [],
    "val_loss": [],
    "train_coefficient": [],
    "val_coefficient": [],
    "learning_rates": [],
    "epoch_times": []
}


for epoch in range(epochs):
    start_time = time.time()
    model.train()
    Loss_train = 0 
    coeff_train = 0

    with tqdm(train_loader, unit="batch") as trainer:
        for batch in trainer:
            trainer.set_description("Train")
            train_feat = batch[:, 0:-1].to(device)
            train_labs = batch[:, -1].to(device)
            train_pred = model(train_feat).squeeze()
            train_loss = torch.nn.functional.mse_loss(train_pred, train_labs)
            train_coef = coeff_determination(train_pred, train_labs) 

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            Loss_train += train_loss.item()
            coeff_train += train_coef.item()

        Loss_train /= len(trainer)
        coeff_train /= len(trainer)

    model.eval()
    Loss_val = 0
    coeff_val = 0
    with tqdm(val_loader, unit="batch") as validator:
        for batch in validator:
            validator.set_description("Valid")
            val_feat = batch[:, 0:-1].to(device)
            val_labs = batch[:, -1].to(device)
            val_pred = model(val_feat).squeeze()
            val_loss = torch.nn.functional.mse_loss(val_pred, val_labs)
            val_coef = coeff_determination(val_pred, val_labs) 

            Loss_val += val_loss.item()
            coeff_val += val_coef.item()

        Loss_val /= len(validator)
        coeff_val /= len(validator)
    
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    history["learning_rates"].append(current_lr)
    
    epoch_duration = time.time() - start_time

    history["train_loss"].append(Loss_train)
    history["val_loss"].append(Loss_val)
    history["train_coefficient"].append(coeff_train)
    history["val_coefficient"].append(coeff_val)
    history["learning_rates"].append(optimizer.param_groups[0]['lr'])
    history["epoch_times"].append(epoch_duration)

    if early_stopper.early_stop(model.state_dict(), Loss_val):             
        print("Early stopping...")
        break
    
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {Loss_train:.4f}, Train Coeff: {coeff_train:.4f}, "
          f"Val Loss: {Loss_val:.4f}, Val Coeff: {coeff_val:.4f}, Time: {epoch_duration:.2f} sec")

    
with open(f"training_history_{dt_name}.json", "w") as f:
    json.dump(history, f)
    
print(f"Training history saved to 'training_history_{dt_name}.json'")


Valid: 100%|██████████| 138/138 [00:05<00:00, 26.06batch/s]


Epoch 1/6000, Train Loss: 0.8527, Train Coeff: -1754.2955, Val Loss: 0.5737, Val Coeff: -0.9258, Time: 25.82 sec


Valid: 100%|██████████| 138/138 [00:05<00:00, 26.18batch/s]


Epoch 2/6000, Train Loss: 0.5470, Train Coeff: -0.3519, Val Loss: 0.4611, Val Coeff: -0.4175, Time: 25.79 sec


Valid: 100%|██████████| 138/138 [00:05<00:00, 25.79batch/s]


...

Epoch 509/6000, Train Loss: 0.3611, Train Coeff: 0.4009, Val Loss: 0.3499, Val Coeff: 0.3183, Time: 25.71 sec


Valid: 100%|██████████| 138/138 [00:05<00:00, 25.83batch/s]


Epoch 510/6000, Train Loss: 0.3609, Train Coeff: 0.3974, Val Loss: 0.3514, Val Coeff: 0.3103, Time: 25.79 sec


Valid: 100%|██████████| 138/138 [00:05<00:00, 25.82batch/s]


+++ Early Stopping is reached! +++
Early stopping...
Training history saved to 'training_history_M4_103.json'


In [ ]:
data_iter = iter(train_loader)
next(data_iter)[:,0:-1]

traced_script_module = torch.jit.trace(model, next(data_iter)[:,0:-1].to(device))
traced_script_module.save(f"traced_model_{dt_name}.pt")